In [1]:
import os
os.listdir("/kaggle/input")

['titanic-dataset']

In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/titanic-dataset/Titanic-Dataset.csv")
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
df["Survived"].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [6]:
# Age column मधले missing values mean ने भरतो
df["Age"].fillna(df["Age"].mean(), inplace=True)

# Embarked column मधले missing values mode ने भरतो
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)

/tmp/ipykernel_17/2850547062.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].mean(), inplace=True)
/tmp/ipykernel_17/2850547062.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [7]:
df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

In [8]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [9]:
# Sex column convert
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

# Embarked column one-hot encoding
df = pd.get_dummies(df, columns=["Embarked"], drop_first=True)

In [10]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,False,True
1,1,1,1,38.0,1,0,71.2833,False,False
2,1,3,1,26.0,0,0,7.9250,False,True
3,1,1,1,35.0,1,0,53.1000,False,True
4,0,3,0,35.0,0,0,8.0500,False,True


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Features आणि Target
X = df.drop("Survived", axis=1)
y = df["Survived"]

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prediction
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8100558659217877

In [12]:
df["Fare"].fillna(df["Fare"].mean(), inplace=True)

/tmp/ipykernel_17/4124385863.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Fare"].fillna(df["Fare"].mean(), inplace=True)


In [13]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df.drop("Survived", axis=1)
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy



0.8100558659217877

In [15]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [16]:
# Sex column convert
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

# Embarked column convert
df["Embarked"] = df["Embarked"].map({"S": 0, "C": 1, "Q": 2})

KeyError: 'Embarked'

In [ ]:
for col in df.columns:
    if df[col].dtype != "object":
        df[col].fillna(df[col].mean(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

In [ ]:
df.isnull().sum()

In [ ]:
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df.drop("Survived", axis=1)
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/titanic-dataset/Titanic.csv')

In [ ]:
# Missing values fill
df['Age'].fillna(df['Age'].mean(), inplace=True)
df['Fare'].fillna(df['Fare'].mean(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Sex encoding
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Embarked one-hot encoding
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/titanic-dataset/Titanic.csv')

In [ ]:
import os
os.listdir('/kaggle/input')

In [ ]:
os.listdir('/kaggle/input/titanic')

In [ ]:
import os
os.listdir('/kaggle/input')

In [ ]:
os.listdir('/kaggle/input/titanic')

In [ ]:
import os
os.listdir('/kaggle/input')

In [ ]:
os.listdir('/kaggle/input/titanic-dataset')

In [ ]:
os.listdir('/kaggle/input')

In [ ]:
import os
os.listdir('/kaggle/input/titanic-dataset')

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/titanic-dataset/Titanic.csv')
df.head()

In [ ]:
df = pd.read_csv('/kaggle/input/titanic-dataset/train.csv')
df.head()


In [ ]:
import os
os.listdir('/kaggle/input/titanic-dataset')

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/titanic-dataset/Titanic.csv')
df.head()


In [ ]:
df = pd.read_csv('/kaggle/input/titanic-dataset/titanic.csv')
df.head()


In [ ]:
import os
os.listdir('/kaggle/input/titanic-dataset')

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/titanic-dataset/Titanic Dataset.csv')
df.head()


In [ ]:
import os
os.listdir('/kaggle/input')
os.listdir('/kaggle/input/titanic-dataset')


In [ ]:
df.head()


In [ ]:
df.columns


In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
